In [ ]:
#import sys
#!{sys.executable} -m pip install futures
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
import pandas as pd
from time import time
from concurrent.futures import ThreadPoolExecutor
from scipy.optimize import curve_fit
with open("+The jungle book_Kipling.txt") as f:
    file=f.read()
 

print(file)

In [56]:
import sys
!{sys.executable} -m pip install plotly



  Using cached plotly-4.14.3-py2.py3-none-any.whl (13.2 MB)
  Using cached retrying-1.3.3-py3-none-any.whl


In [3]:
from string import punctuation
def remove_punctuation(data):
    temp=[]
    for i in data:
        if i in punctuation:
            if i == "'" or i == '-'or i==' ':
                temp.append(i.lower())
        else:
            temp.append(i.lower())
    return "".join(temp)
data=remove_punctuation(file)

print(data.split())

['\ufeffthe', 'jungle', 'book', 'contents', "mowgli's", 'brothers', 'hunting-song', 'of', 'the', 'seeonee', 'pack', "kaa's", 'hunting', 'road-song', 'of', 'the', 'bandar-log', 'tiger', 'tiger', "mowgli's", 'song', 'the', 'white', 'seal', 'lukannon', 'rikki-tikki-tavi', "darzee's", 'chant', 'toomai', 'of', 'the', 'elephants', 'shiv', 'and', 'the', 'grasshopper', 'her', "majesty's", 'servants', 'parade', 'song', 'of', 'the', 'camp', 'animals', "mowgli's", 'brothers', 'now', 'rann', 'the', 'kite', 'brings', 'home', 'the', 'night', 'that', 'mang', 'the', 'bat', 'sets', 'free--', 'the', 'herds', 'are', 'shut', 'in', 'byre', 'and', 'hut', 'for', 'loosed', 'till', 'dawn', 'are', 'we', 'this', 'is', 'the', 'hour', 'of', 'pride', 'and', 'power', 'talon', 'and', 'tush', 'and', 'claw', 'oh', 'hear', 'the', 'call--good', 'hunting', 'all', 'that', 'keep', 'the', 'jungle', 'law', 'night-song', 'in', 'the', 'jungle', 'it', 'was', 'seven', "o'clock", 'of', 'a', 'very', 'warm', 'evening', 'in', 'the', 

In [4]:

class Ngram (dict):
    def __init__ (self, iterable=None): # Ініціалізували наш розподіл як новий об'єкт класу, додаємо наявні елементи
        super(Ngram, self).__init__()
        self.F_i = 0 # число унікальних ключів в розподілі
        self.fa={}
        self.counts={}
        if iterable:
           self.update (iterable)
    def update (self, iterable): # Оновлюємо розподіл елементами з наявного итерируемого набору даних
        for item in iterable:
           if item in self:
               self[item]+=1
           else:
                self[item] = 1
                self.F_i+=1
    def hist(self):
        plt.bar(self.keys(),self.values())
        plt.show()

def make_dataframe(model,fmin):
    filtred_data=list(filter(lambda x:model[x].F_i >=fmin,model))
    data={"ngram":[],
          "F_i":np.empty(len(filtred_data),dtype=np.dtype(int))}
    
    #print(data['ngram'][0])
    for i,ngram in enumerate(filtred_data):
        data["ngram"].append(ngram)
        data["F_i"][i]=len(model[ngram].pos)
        
    return pd.DataFrame(data=data)
L=0#
V=0#
def make_markov_chain(data,order=1,split='word'):
    model=dict()
    
    if split=="symbol":
        temp=[]
        for word in data:
            for symbol in word: 
                temp.append(symbol)
        data=temp
    global L
    L=len(data)-order
    if order>1:
        for i in range(L):
            window = tuple (data[i: i+order])  # Додаємо в словник
            if window in model: # Приєднуємо до вже існуючого розподілу
                 model[window].update ([data[i+order]])
                 model[window].pos.append(i+1)
                 model[window].bool[i]=1
                
            else:
                 model[window] = Ngram ([data[i+order]])
                 model[window].pos=[]
                 model[window].pos.append(i+i)
                 model[window].bool=np.zeros(len(data)-order)
                 model[window].bool[i]=1
    else:
        for i in range(L):
            if data[i] in model: # Приєднуємо до вже існуючого розподілу
                 model[data[i]].update ([data[i+order]])
                 model[data[i]].pos.append(i+order)
                 model[data[i]].bool[i]=1
            else:
                 model[data[i]] = Ngram ([data[i+order]])
                 model[data[i]].pos=[]
                 model[data[i]].pos.append(i+1)
                 model[data[i]].bool=np.zeros(len(data)-order)
                 model[data[i]].bool[i]=1
    global V
    V=len(model)
    return model
start=time()
fmin=10
order=2
split="word"
model=make_markov_chain(data.split(),order=order,split=split)

#model
df=make_dataframe(model,fmin)
print("chain time:",time()-start)
print(df)


chain time: 0.18953275680541992
                ngram  F_i
0           (of, the)  121
1   (the, bandar-log)   16
2          (and, the)   28
3       (the, jungle)   66
4           (in, the)   80
5           (it, was)   18
6             (of, a)   15
7      (father, wolf)   26
8      (mother, wolf)   16
9         (into, the)   22
10           (it, is)   23
11          (he, was)   27
12        (a, little)   19
13          (with, a)   14
14        (from, the)   15
15        (and, then)   18
16         (then, he)   12
17          (in, his)   15
18           (is, no)   11
19           (for, a)   10
20          (to, the)   36
21          (he, had)   23
22      (shere, khan)   43
23         (for, the)   24
24          (he, has)   22
25         (that, is)   14
26          (and, he)   21
27         (when, he)   16
28           (he, is)   28
29         (all, the)   17
30          (of, his)   16
31           (was, a)   11
32           (as, he)   20
33        (with, his)   11
34       (man's, cub)  

# Calculate fa:
1.model[ngram].bool --> [0,0,0,0,0,1,1,0,1,...L]
2.windows(w,we,wh,wmax)

In [ ]:
print("order:",order)
print("split by:",split)
print("L:",L)
print("V: ",V)
print("fmin:",fmin)
print("valid V:",len(df['ngram']))
print()
wmax=int(L/20)
w=int(wmax/10)
we=10
wh=w
print("w:",w)
print("wmax:",wmax)
print("we:",we)
print("wh:",wh)


opt ="no"
print("option:",opt)
#model['entropy'].bool

## calculate distance (boundary condition)

In [ ]:
@jit(nopython=True)
def nbc(positions):
    number_of_pos=len(positions)
    if number_of_pos ==1:
        return positions
    dt=np.empty(number_of_pos-1,dtype=np.uint8)
    for i in range(number_of_pos-1):
        dt[i]=positions[i+1]-positions[i]
    return dt

@jit(nopython=True)
def obc(positions,L):
    number_of_pos=len(positions)
    dt=np.empty(number_of_pos+1,dtype=np.uint8)
    dt[0]=positions[0]
    for i in range(number_of_pos-1):
        dt[i+1]=positions[i+1]-positions[i]
    dt[-1]=L-positions[-1]
    return dt

@jit(nopython=True)
def pbc(positions,L):
    number_of_pos=len(positions)
    dt=np.empty(number_of_pos,dtype=np.uint8)
    for i in range(number_of_pos-1):
        dt[i]=positions[i+1]-positions[i]
    dt[-1]=L-positions[-1]+L+positions[0]
    return dt


for index,ngram in enumerate(df['ngram']):
    print(str(index)+" of "+str(len(df['ngram'])),end="\r")
    model[ngram].dt=nbc(np.array(model[ngram].pos,dtype=np.uint8),L)
    #print(model[ngram].dt)
print(1)

In [ ]:
start=time()
temp_w=0


#print(df["ngram"])
@jit(nopython=True)
def s(window):
    suma=0
    for i in range(len(window)):
        suma+=window[i]
    return suma
@jit(nopython=True)
def mse(x):
    t=np.mean(x)
    st=np.mean(x**2)
    return np.sqrt(st-(t**2))

@jit(nopython=True)
def R(x):
    t=np.mean(x)
    if t == 0:
        return 0
    ts=np.mean(x**2)
    return np.sqrt(ts-(t**2))/t
    
@jit(nopython=True)
def fa(x,args):
    #print(w)
    wi,wsh,l=args
    
    count=np.empty(len(range(wi,l,wsh)),dtype=np.uint8)

    for index,i in enumerate(range(0,l-wi,wh)):
        count[index]=s(x[i:i+wi])
        
    return count,mse(count)
windows=list(range(w,wmax,we))
fa(np.zeros(5),(1,2,3))

def func(wind):
    model[ngram].counts[wind],model[ngram].fa[wind]=fa(model[ngram].bool,(wind,wh,L))

for index,ngram in enumerate(df['ngram']):
    print(str(index)+" of "+str(len(df['ngram'])),end="\r")
    with ThreadPoolExecutor() as e:
        e.map(func,windows)
@jit(nopython=True)
def fit(x,a,b):
    return a*x**b
temp_b=[]
temp_fi=[]
temp_R=[]
temp_cov=[]
for ngram in df['ngram']:  
    c,cov=curve_fit(fit,[*model[ngram].fa.keys()],[*model[ngram].fa.values()],maxfev=5000)
    model[ngram].a=c[0]
    model[ngram].b=c[1]
    temp_b.append(c[1])
    temp_cov.append(cov)
    #temp_fi.append(model[ngram].F_i/L)
    temp_R.append(R(np.array(model[ngram].dt)))
df['R']=temp_R
df["cov"]=temp_cov
#df['f_i']=temp_fi
df['alpha']=temp_b

    

    

    
"""
def secound_pool(ngram):
    return fa(model[ngram].bool,(temp_w,wmax,we,wh,L))

def first_pool(wind):
    #print(w)
    #print(list(df['ngram']))
    #print(model['entropy'])
    global temp_w
    temp_w=wind
    with ThreadPoolExecutor() as j:
        result = j.map(secound_pool,df['ngram'])
    for f,ngram in zip(result,df['ngram']):
        model[ngram].counts[wind]=f[0]
        model[ngram].fa[wind]=f[1]

    
with ThreadPoolExecutor() as e:
    windows=list(range(w,wmax,we))
    e.map(first_pool,windows)
"""


print("fa time:",time()-start)
df

# Graphs
    


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
#fig = px.scatter(
#    df, x='total_bill', y='tip', opacity=0.65,
#    trendline='ols', trendline_color_override='darkblue'
#)
#fig.show()
delta_f=[]
ngram=('of','the','pack')
for w in model[ngram].fa.keys():
    delta_f.append(fit(w,model[ngram].a,model[ngram].b))

fig=go.Figure(
    [
        go.Scatter(x=[*model[ngram].fa.keys()],y=[*model[ngram].fa.values()],mode="markers"),
        go.Scatter(x=[*model[ngram].fa.keys()],y=delta_f)
        
    ]

)
fig.update_xaxes(type="log")
fig.show()
#print(len(np.array(model[ngram].pos)))
#print(len(model[ngram].bool))
fig1=go.Figure(
    [
        go.Scatter(x=np.array(range(L)),y=model[ngram].bool)
    ]
)
#fig1.update_layout(barmode='group')
fig1.show()


# tests
order: 1
split by: word
L: 479154
V:  14262
fmin: 15
valid V: 1792

w: 4791
wmax: 47915
we: 1000
wh: 4791

# same window for all
fa time: 321.6836111545563 sec/(5,35 m)

# one by one 
fa time: 210.61292123794556 sec/ 3,5 m